# Interactive Redesign Project

### Alina Faisal (alinafai)

## Improved Visualization 2: Impact of Economic Status on ELA Academic Outcomes (Year 2013)
The scatter plot illustrates the relationship between economic status and academic performance in English Language Arts (ELA) among New York schools in 2013. It shows a strong positive correlation between mean scale scores and proficiency levels, with economically disadvantaged students (blue) generally scoring lower than their more affluent peers (red). A concentration of blue dots at lower proficiency levels highlights the academic challenges faced by disadvantaged students, while overlap at higher scores suggests that some students from low-income backgrounds still achieve high proficiency. The visualization underscores the persistent educational disparities linked to socioeconomic status.
Source: https://infohub.nyced.org/

In [7]:
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [8]:
import pandas as pd
import altair as alt

alt.data_transformers.enable("json")

file_path = "/Users/alinafaisal/Desktop/UMich/Spring25/SI649/static viz project/data/ny-ela-results-2013-2019-public-master.xlsx"

# Economic Status Data Processing
use_cols_econ = ["Year", "Category", "Mean Scale Score", "% Level 3+4", "Number Tested"]
ela_econ_df = pd.read_excel(file_path, sheet_name="Econ Status", usecols=use_cols_econ)

# Filter for Years 2013-2015
ela_econ_filtered = ela_econ_df[ela_econ_df["Year"].isin([2013, 2014, 2015])].copy()
ela_econ_filtered.replace("s", None, inplace=True)
ela_econ_filtered.dropna(inplace=True)

# Convert columns to numeric
ela_econ_filtered["Mean Scale Score"] = pd.to_numeric(ela_econ_filtered["Mean Scale Score"], errors="coerce")
ela_econ_filtered["% Level 3+4"] = pd.to_numeric(ela_econ_filtered["% Level 3+4"], errors="coerce")

# Limit dataset size for Altair compatibility
ela_econ_filtered = ela_econ_filtered.sample(n=min(4000, len(ela_econ_filtered)), random_state=42)

### Ethnicity Data Processing
use_cols_ethnicity = ["Year", "Category", "Mean Scale Score"]
ela_ethnicity_df = pd.read_excel(file_path, sheet_name="Ethnicity", usecols=use_cols_ethnicity)

# Filter for Years 2013-2015
ela_ethnicity_filtered = ela_ethnicity_df[ela_ethnicity_df["Year"].isin([2013, 2014, 2015])].copy()
ela_ethnicity_filtered.replace("s", None, inplace=True)
ela_ethnicity_filtered.dropna(inplace=True)

# Convert columns to numeric
ela_ethnicity_filtered["Mean Scale Score"] = pd.to_numeric(ela_ethnicity_filtered["Mean Scale Score"], errors="coerce")

# Limit dataset size for Altair compatibility
ela_ethnicity_filtered = ela_ethnicity_filtered.sample(n=min(4000, len(ela_ethnicity_filtered)), random_state=42)

### Dropdown Selection for Year
selectYear = alt.selection_point(
    fields=["Year"],
    bind=alt.binding_select(options=[2013, 2014, 2015], name="Select Year: ")
)

brush = alt.selection_interval(encodings=["x"])  # Zooming Brush

# Annotation Data
annotation_data = pd.DataFrame({
    "Mean Scale Score": [270, 320, 350],
    "% Level 3+4": [30, 55, 80],
    "Text": ["Low Performance Cluster", "Mid Performance Cluster", "High Performance Cluster"]
})

annotations = alt.Chart(annotation_data).mark_text(
    align="center", baseline="bottom", fontSize=14, fontWeight="bold", color="black"
).encode(
    x="Mean Scale Score:Q",
    y="% Level 3+4:Q",
    text="Text:N"
)

# Scatter Plot
scatter_plot = alt.Chart(ela_econ_filtered).transform_filter(selectYear).mark_circle(
    size=80, opacity=0.6, stroke='white', strokeWidth=0.5
).encode(
    x=alt.X("Mean Scale Score:Q", title="Mean Scale Score (Academic Performance)", scale=alt.Scale(domain=[230, 370])),
    y=alt.Y("% Level 3+4:Q", title="Percentage of Students Proficient (Level 3+4)", scale=alt.Scale(domain=[0, 100])),
    color=alt.Color(
        "Category:N",
        title="Economic Status",
        legend=alt.Legend(
            orient="right",
            titleFontSize=14,  
            titleLimit=250,  
            labelLimit=200  
        )
    ),
    tooltip=["Year", "Category", "Mean Scale Score", "% Level 3+4"]
).properties(width=400, height=300, title="Economic Status & ELA Outcomes (2013-2015)").add_params(selectYear, brush)

scatter_plot = scatter_plot + annotations 

# Line Chart
line_chart = alt.Chart(ela_econ_filtered).transform_filter(selectYear).transform_filter(brush).mark_line().encode(
    x=alt.X(
        "Mean Scale Score:Q",
        title="Mean Scale Score (Academic Performance)"
    ),
    y=alt.Y(
        "% Level 3+4:Q",
        title="Percentage of Students Proficient (Level 3+4)",
        scale=alt.Scale(domain=[0, 100])
    ),
    color=alt.Color(
        "Category:N",
        title="Economic Status",
        legend=alt.Legend(
            orient="right",
            titleFontSize=14,
            titleLimit=250,
            labelLimit=200
        )
    )
).properties(width=400, height=300, title="Trends in Student Proficiency by Economic Status")

# Bar Chart (Ethnicity Mean Scale Scores)
ethnicity_bar_chart = alt.Chart(ela_ethnicity_filtered).mark_bar(opacity=0.8).encode(
    y=alt.Y("Category:N", title="Ethnicity Group"),
    x=alt.X("Mean Scale Score:Q", title="Mean Scale Score (Ethnicity)"),
    color=alt.Color("Category:N", title="Ethnicity Group"),
    column=alt.Column("Year:N", title="Year"),
    tooltip=["Year", "Category", "Mean Scale Score"]
).properties(width=300, height=150, title="Mean Scale Scores by Ethnicity (2013-2015)").add_params(selectYear).transform_filter(selectYear)

# Combining All Visualizations
final_chart = alt.vconcat(
    alt.hconcat(scatter_plot, line_chart),  # Economic Status Plots
    ethnicity_bar_chart,  # Ethnicity Bar Chart Below
).configure_view(strokeWidth=0)

# Display Final Chart
final_chart

alt.VConcatChart(...)

## Improved Visualization 4: Segregation rates and Free/Reduced Lunch Students in US Public Schools (Years 2015-2016)

In [9]:
import pandas as pd
import altair as alt
from vega_datasets import data

# Load the dataset
file_path = "/Users/alinafaisal/Desktop/UMich/Spring25/SI649/static viz project/001-alinafai-AlinaFaisal/NCES_public_clean.csv"
df = pd.read_csv(file_path)

# Clean column names (strip spaces)
df.columns = df.columns.str.strip()

# Rename relevant columns for easier access
df = df.rename(columns={
    "State Name [Public School] 2015-16": "State",
    "Free and Reduced Lunch Students [Public School] 2015-16": "Free/Reduced Lunch Students",
    "White Students [Public School] 2015-16": "White_Students",
    "Total Students All Grades (Excludes AE) [Public School] 2015-16": "Total_Students",
    "Longitude [Public School] 2015-16": "Longitude",
    "Latitude [Public School] 2015-16": "Latitude",
    "School Name": "School_Name"
})

# Standardize State Names
df["State"] = df["State"].str.strip().str.title()

# Convert columns to numeric
df["Free/Reduced Lunch Students"] = pd.to_numeric(df["Free/Reduced Lunch Students"], errors="coerce")
df["White_Students"] = pd.to_numeric(df["White_Students"], errors="coerce")
df["Total_Students"] = pd.to_numeric(df["Total_Students"], errors="coerce")

# Compute percentage of white students
df["Percentage_White_Students"] = (df["White_Students"] / df["Total_Students"]) * 100

# Compute intensely segregated indicator (1 if <10% white, else 0)
df["Intensely_Segregated"] = df["Percentage_White_Students"].apply(lambda x: 1 if x < 10 else 0)

# Compute segregation rate per state
state_segregation = df.groupby("State")[["Intensely_Segregated"]].sum()
state_counts = df.groupby("State")[["School_Name"]].count()
state_segregation["Segregation_Rate"] = state_segregation["Intensely_Segregated"] / state_counts["School_Name"]
state_segregation = state_segregation.reset_index()

# Compute Free/Reduced Lunch Students per state
state_lunch = df.groupby("State")[["Free/Reduced Lunch Students"]].sum().reset_index()

# Merge DataFrames
merged_data = state_segregation.merge(state_lunch, on="State")

# Map State Names to FIPS Codes
state_fips = pd.DataFrame({
    "State": [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
        "Connecticut", "Delaware", "District Of Columbia", "Florida", "Georgia",
        "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
        "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
        "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
        "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota",
        "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
        "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia",
        "Washington", "West Virginia", "Wisconsin", "Wyoming"
    ],
    "id": [
        1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23,
        24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
        44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56
    ]
})

# Merge FIPS codes
merged_data = merged_data.merge(state_fips, on="State", how="left")

# **Debugging Step: Check for missing FIPS codes**
missing_fips = merged_data[merged_data["id"].isna()]
if not missing_fips.empty:
    print("⚠️ States with missing FIPS codes:")
    print(missing_fips)

# Convert id to integer (handling NaNs first)
merged_data["id"] = merged_data["id"].fillna(0).astype(int)

# Load US states geometry
states = alt.topo_feature(data.us_10m.url, feature="states")

# Interactive Selection Mechanism (Click-based Selection)**
# Create selection for interaction
selection = alt.selection_point(fields=['State'], empty="all", clear=True)

#US Map with Interactive Selection**
choropleth = alt.Chart(states).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(merged_data, 'id', ['State', 'Segregation_Rate'])
).encode(
    color=alt.Color('Segregation_Rate:Q', scale=alt.Scale(scheme="yellowgreenblue"), title="Segregation Rate"),
    tooltip=['State:N', 'Segregation_Rate:Q'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.3))  # Selected state fully opaque, others faded
).add_params(selection).project(
    type="albersUsa"
).properties(
    width=600,
    height=400,
    title="Segregation Rates by U.S. States (2015-2016)"
)

#Bar Chart for Free/Reduced Lunch Students (Click to highlight state)**
bar_chart = alt.Chart(merged_data).transform_window(
    rank="rank()",
    sort=[alt.SortField("Free/Reduced Lunch Students", order="descending")]
).transform_filter(
    alt.datum.rank <= 20
).mark_bar().encode(
    y=alt.Y("State:N", sort="-x", title="State"),
    x=alt.X("Free/Reduced Lunch Students:Q", title="Free/Reduced Lunch Students",
        axis=alt.Axis(format=",.0f")),
    color=alt.Color("Free/Reduced Lunch Students:Q", scale=alt.Scale(scheme="blues")),
    tooltip=["State:N", "Free/Reduced Lunch Students:Q"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.3))  
).properties(
    width=600,
    height=400,
    title="Top 20 U.S. States by Free/Reduced Lunch Students (2015-2016)"
).add_params(selection)

# Compute dynamically filtered mean of selected Free/Reduced Lunch Students
mean_agg = alt.Chart(merged_data).transform_filter(
    selection
).transform_aggregate(
    avg_selected="mean(Free/Reduced Lunch Students)"
)

# Dynamic Mean Line (Moves Based on Selection)
mean_line = mean_agg.mark_rule(color="firebrick", size=3).encode(
    x="avg_selected:Q"
)

# Dynamic Mean Score Text Annotation (Moves with the Line)
mean_text = mean_agg.mark_text(
    align="left", dx=5, dy=-5, color="firebrick", fontSize=12
).encode(
    x="avg_selected:Q",
    text=alt.Text("avg_selected:Q", format=",d")  # Formatting to include commas
)

# Dynamic Label for "Average Value" (Moves with the Line)
mean_label = mean_agg.mark_text(
    align="left", dx=5, dy=-20, color="firebrick", fontSize=12, fontWeight="bold"
).encode(
    x="avg_selected:Q",
    text=alt.value("Students")  # Static text that moves
)

# Add these to the final bar chart so they move dynamically
bar_chart = (bar_chart + mean_line + mean_text + mean_label)

#Combine US Map & Interactive Bar Chart with Mean Line
final_chart = (choropleth | (bar_chart + mean_line + mean_text)).resolve_scale(color='independent')

#Display
final_chart

# Define caption text
caption_text = """Figure. Connection between segregation rates (map) and free/reduced-price lunch recipients (bar chart)
highlights racial and economic disparities in U.S. education. Schools with higher segregation in schools - fewer
than 10% white students - often have higher concentrations of economically disadvantaged students,
reflecting systemic inequities that perpetuate poverty and educational gaps.
"""

# Convert caption text into a DataFrame for Altair
caption_df = pd.DataFrame({"text": caption_text.split("\n")})

caption = alt.Chart(pd.DataFrame({'text': [caption_text]})).mark_text(
    align='left',  # Ensure left alignment
    baseline='top',  # Align to the top
    dx=-350,  # Shift the text further left (adjust as needed)
    fontSize=12, 
    color='black'
).encode(
    text='text:N'
).properties(
    width=100, height=50
)

# Stack the caption **below** the visualization
final_chart_with_caption = alt.vconcat(final_chart, caption).configure_view(
    strokeWidth=0  # Removes extra borders
)

# Apply title configuration separately
final_chart_with_caption = final_chart_with_caption.configure_title(
    anchor="middle",  # Align title to the left
    fontSize=14
)

# Display the final visualization with the properly formatted caption
final_chart_with_caption



alt.VConcatChart(...)